# Running Certifai Pro scans on Azure ML models

## CONTENTS

1. First we'll understand how to install Certifai Pro and its setup in an Azure cloud environment. Setup process includes configuring your Certifai Pro instance with storage parameters for a pre-existing container in an Azure Storage Account

2. We will then install the required Certifai Toolkit python libraries so scans that generate information about the fairness, robustness and explainability of your ML models can be evaluated with the [CERTIFAI framework](https://cognitivescale.github.io/cortex-certifai/docs/about)

3. Then, we will fetch credentials for Azure Hosted Model Endpoint Services for sklearn models to classify [german credit loan risk](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29) (predict whether loan will be granted or not). These credentials maybe obtained by following the [Azure ML Certifai Scan Notebook](./german_credit_azure_ml_demo.ipynb)

5. Test the deployed Hosted Model Endpoint Services (Webservices)

6. Construct Certifai Scan Definitions for this Binary Classification model

7. Upload the required datasets for the scan into the Azure Storage Account container used to configure Certifai Pro on Step 1

8. Configure the Certifai CLI with a Kubeconfig for the Certifai pro instance created in step 1

9. Submit a remote scan job to the Certifai Pro instance through the Certifai CLI using the Scan definitions constructed in Step 6

<a id='prereqs'> - Certifai Pro

Cortex Certifai Pro is a cloud based offering from [CognitiveScale](https://www.cognitivescale.com/certifai/) that allows data scientists to define, scan and analyse their models to determine their Fairness, Robustness and Explainability measures using the [CERTIFAI framework](https://arxiv.org/abs/1905.07857). 

You can find more details about Certifai on the [official documentation site](https://cognitivescale.github.io/cortex-certifai/docs/about).

This tutorial helps users that use **Azure Machine Learning resources (Hosted Notebooks/Models/Endpoints)** to setup their model and ready it for scanning with Certifai Pro.

Certifai Pro is a single user, VM installed version of Cortex Certifai that runs on an Azure VM and can be installed from the [Azure Marketplace](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/cognitive-scale.cortex-certifai-pro). Certifai Pro VMs can run scans on your own models with the aid of the Certifai Toolkit, a downloadable set of Python packages and CLI tools that can run Certifai scans on your personal machines. The toolkit also enables you to connect to and run scans remotely on the Certifai Pro VM. You can find more details about the [Certifai Toolkit](https://cognitivescale.github.io/cortex-certifai/docs/toolkit/setup/download-toolkit)

This guide walks you through using the Certifai Python API to help define Scan Definitions that can be passed on to the Certifai Pro instance along with Datasets and Secrets needed, if any
be covered separately


### Install Certifai Pro from the Azure Marketplace

You can find and create a personal instance of Cortex Certifai Pro from the [Azure Marketplace](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/cognitive-scale.cortex-certifai-pro). Please follow the instructions from the official [Certifai docs](https://cognitivescale.github.io/cortex-certifai/docs/platforms/azure/azure-setup) for the Azure platform to get up and running.

A brief summary of this process includes:
- Certifai Pro instance setup from the Marketplace and initial authentication workflows
- Configure your Certifai Pro instance with blob storage containers and credentials for an Azure Storage account of your choice.
    - You may also install sample reports for a variety of usecases in Finance, Healthcare and Insurance to understand how the AI Trust Index scores generated by Certifai, accompanied by the extraordinarily helpful reports on Fairness, Robustness and Explainability can be used to improve your machine learning models
- Configure Custom SSL certificates (if needed)


### Certifai Pro Install and Setup

Follow the instructions described in the [Azure Storage Setup](https://cognitivescale.github.io/cortex-certifai/docs/platforms/azure/azure-setup#certifai-console-storage-setup) to configure your Azure based Certifai Console hosted via the Certifai Pro VM with an Azure Blob Container Storage Account of your choice.

When configuring the storage parameters for your Azure Certifai Pro instance, please make note of the `Scan Directory` field you've used to configure your instance. This field will be used later on in this tutorial to store the reports of the scan whose definition we will construct in thsi notebook and run remotely on the Certifai Pro instance.

You can also uncheck the `Install Sample Reports` checkbox on the `Storage Settings` page of the Certifai Pro instance setup. You do however need to enable the `Download Kubeconfig` option

So, please go ahead to the Azure Marketplace listing for [Cortex Certifai Pro](https://azuremarketplace.microsoft.com/en-us/marketplace/apps/cognitive-scale.cortex-certifai-pro) and setup your instance of Certifai Pro and head back here after you've completed the detailed instructions we've provided for Azure on the [official Certifai docs](https://cognitivescale.github.io/cortex-certifai/docs/platforms/azure/azure-setup).


Once you're done, head back to this tutorial where we walk you through the widely known [German Credit dataset](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29) and build two classification models that determine whether a Bank should extend lines of Credit to customers based on demographic, financial and employment features.

Both of the classification models (SVM and Logistic Regression) we'll build today use the `Scikit-learn` Python module. You can follow along to get a good sense of the contracts/patterns needed to deploy a machine learning model that your Certifai Pro installation understands and connects to (for inference).

The following notebook cells walk you through the routine Data Science workflow (Pre-process, Data Splits, Model Training and Model Deployment). Feel free to use these notebooks as a starting point in your journey to scan your machine learning models with the CERTIFAI framework and gain a deeper understanding of their performance and behavior in terms of the following quantities:

    1. Robustness to Data Variations
    2. Explainability of Model Predictions
    3. Fairness By Group

Refer to the [Certifai Quickstart](https://cognitivescale.github.io/cortex-certifai/docs/quickstart) for a rehash on each of these topics and what they mean in general and what they can mean to your machine learning model.

### Download the Cortex Certifai Toolkit from your Certifai Pro VM

Follow the instructions described in [Certifai Pro Azure Setup](https://cognitivescale.github.io/cortex-certifai/docs/platforms/azure/azure-setup) to finish initial setup for your Certifai Pro VM. Now, click on the Help icon (top right) and select `Download Toolkit` to download a zip file containing the Cortex Certifai Toolkit to your computer.

If you're running this notebook from an Azure Hosted Notebook, you'll need to upload the Certifai Toolkit zip file to the Hosted Notebook and make note of its path.

## Prerequisites - Notebook Dependencies

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the 
[configuration-notebook](https://github.com/Azure/MachineLearningNotebooks/blob/c520bd1d4130d9a01ee46e0937459e2de95d15ec/configuration.ipynb) to create an Azure workspace. Creating local and remote environments/dependencies will be covered in the notebook

**PleaseNote**: to step through this notebook, make sure you have necessary dependencies installed locally

- python>=3.6.2,<3.7
- ipython
- matplotlib
- jupyter

You can also use [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/) to create the local environment using the `certifai_azure_model_env.yml` file provided with the notebook

Open your favorite terminal and cd into folder where this notebook is located to execute the below commands

- `jupyter-notebook` : to launch jupyter notebook sesssion. 


**Note**: Installing `Cortex-Certifai` packages will be covered separately

## Certifai Pro on Azure

First, we follow the instructions detailed on the official [Cortex Certifai documentation site](https://cognitivescale.github.io/cortex-certifai/docs/platforms/azure/azure-setup) to tick off the following items from our Pre-requisites:

- [ ] [Install Certifai Pro](https://cognitivescale.github.io/cortex-certifai/docs/platforms/azure/azure-setup) from the Azure Marketplace into an Azure Virtual Machine
- [ ] [Configure](https://cognitivescale.github.io/cortex-certifai/docs/platforms/azure/azure-setup#certifai-console-storage-setup) your Certifai Pro instance with storage credentials for a blob container inside an Azure Storage Account.

Once you've configured your Certifai Pro instance with the Azure Storage Account credentials, download the Certifai Toolkit by clicking on the Help Icon on the top left of the site and selecting `Download Toolkit`
- [ ] Download the Certifai Toolkit and upload it into your hosted Azure ML Notebook and make note of the path

### Set Cortex Certifai Toolkit path
- update the `certifai_toolkit_path` to point to your downloaded Certifai Toolkit
- this will be used later to install cortex certifai python packages

In [1]:
from os.path import expanduser, isfile
home = expanduser("~")
certifai_toolkit_path = f'{home}/Downloads/toolkit'
certifai_toolkit_path

'/Users/pkandarpa/Downloads/toolkit'

## Configure Hosted Model Service Endpoint details.

### Prerequisites - Hosted Azure German Credit Models

Instructions on how one can create and host sci-kit learn models on Azure ML for the German Credit Dataset are described in detail in the [Azure ML Certifai Scan Notebook](./german_credit_azure_ml_demo.ipynb). We've condensed this entire process so that you can obtain a URL and Service Key for SVM and Logistic Regression Sciki-learn models for the current problem into the `scripts/azure_ml_service_keys.py` script. The script

    1. The Service URI and Key for accessing a Azure hosted SVM model service endpoint
    2. The Service URI and Key for accessing a Azure hosted Logistic Regression model service endpoint

Get the above values by

1. Create an Azure Machine Learning Resource and download the workspace's `config.json` to the same directory where this notebook is located. If you've downloaded it elsewhere, make note of this path.

2. Now run `scripts/azure_ml_service_keys.py` we've provided that helps you train, upload and deploy two ML models for the German Credit dataset to the Azure Machine Learning workspace created in step one. You can use the terminal to invoke ths script like this:

`python scripts/azure_ml_service_keys.py --az_config ./config.json`

Ensure that the above command is invoked from a terminal from `notebooks/azureml_model_headers_demo/` directory.

On a successful run of the script, it generates the URL and Service Keys for two Azure Hosted Deployments - one for the SVM model and one for the Logistic Regression model. Make note of those values and input them in the following cell.

The script also generates a list of the Resources created by it that'll be helpful at the end of this notebook to clean all of them up.

In [3]:
service_logistic_uri  = '<replace_this>'
service_logistic_key  = '<replace_this>'

service_svm_uri       = '<replace_this>'
service_svm_key       = '<replace_this>'

In [4]:
# create json test data sample(from csv)

import json
sample_input = json.dumps({
"payload": {
    "instances": [
        [
            "... < 0 DM",
            6,
            "critical account/ other credits existing (not at this bank)",
            "radio/television",
            1169,
            "unknown/ no savings account",
            ".. >= 7 years",
            4,
            "male : single",
            "others - none",
            4,
            "real estate",
            "> 25 years",
            "none",
            "own",
            2,
            "skilled employee / official",
            1,
            "phone - yes, registered under the customers name",
            "foreign - yes"
        ]
    ]
}
})
sample_input

'{"payload": {"instances": [["... < 0 DM", 6, "critical account/ other credits existing (not at this bank)", "radio/television", 1169, "unknown/ no savings account", ".. >= 7 years", 4, "male : single", "others - none", 4, "real estate", "> 25 years", "none", "own", 2, "skilled employee / official", 1, "phone - yes, registered under the customers name", "foreign - yes"]]}}'

In [5]:
import requests
import json

headers_svm = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {service_svm_key}'          
          }
headers_logistic = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {service_logistic_key}'          
          }

response = requests.post(
    service_svm_uri, data=sample_input, headers=headers_svm)
print('SVM Model Endpoint Inference Test')
print(response.status_code)
print(response.elapsed)
print(response.json())

print('Logistic Regression Model Endpoint Inference Test')
response = requests.post(
    service_logistic_uri, data=sample_input, headers=headers_logistic)
print(response.status_code)
print(response.elapsed)
print(response.json())

SVM Model Endpoint Inference Test
200
0:00:00.611955
{'payload': {'predictions': [1]}}
Logistic Regression Model Endpoint Inference Test
200
0:00:01.508665
{'payload': {'predictions': [1]}}


## Installing Cortex Certifai python packages

initiating a Cortex Certifai scan requires following python packages to be installed in the current local environment

`required-packages`

- cortex-certifai-scanner
- cortex-certifai-engine
- cortex-certifai-common

`optional-packages`

- cortex-certifai-client
- cortex-certifai-console

Download [certifai toolkit](https://www.cognitivescale.com/download-certifai) and follow instructions in the `Readme.md` to install the python-packages in the current environment

### Install required certifai packages (optional packages are left for user to install)


In [6]:
!find $certifai_toolkit_path/packages/all -type f   -name "*client-*"                      | xargs -I % sh -c 'pip install % ' ;
!find $certifai_toolkit_path/packages/all -type f   -name "*common-*"                      | xargs -I % sh -c 'pip install % ' ;

Processing /Users/pkandarpa/Downloads/toolkit/packages/all/cortex-certifai-client-1.2.14-115-g96dc7f43.zip
  Created wheel for cortex-certifai-client: filename=cortex_certifai_client-1.2.14-py3-none-any.whl size=27630 sha256=2bf75023565eb56c0c580fcb8996037dffef2f818567079ac433d70bc17918d4
  Stored in directory: /Users/pkandarpa/Library/Caches/pip/wheels/f4/cd/35/a804cc22089ba137950e962afd191dec45d2e54cd4923e108f
Successfully built cortex-certifai-client
  Attempting uninstall: cortex-certifai-client
    Found existing installation: cortex-certifai-client 1.2.14
    Uninstalling cortex-certifai-client-1.2.14:
      Successfully uninstalled cortex-certifai-client-1.2.14


### Using Cortex Certifai Client python-package to launch a Remote Scan
We use the `certifai` cli tool to list, submit and delete Certifai Scans on our Certifai Pro instance.

#### CLI commands

```
# remove flower brackets if not in a jupyter notebook cell
certifai remote config --file certifai-kubeconfig.json --alias {remote_alias} 
```

In [9]:
remote_alias = 'cpro-az'
!certifai remote config --file certifai-kubeconfig.json --alias {remote_alias}


Checking for access to Kubernetes cluster with context - certifai-pro
Connection to cluster succeeded, found API - v1
Scanner image found - cortex-certifai-scanner:local
Updating alias - cpro-az

Configuration updated from - certifai-kubeconfig.json


### List Scan job on the Certifai Pro remote instance
We use the CLI command to list scan jobs for the configured `remote_alias`
```
certifai remote list -a <remote_alias>
```

In [11]:
!certifai remote list -a {remote_alias}

No certifai scan jobs found running on cluster.


### Build Scan definition for our models

We'll now use the `certifai-scanner` python package to build a scan definition for the SVM and Logistic Regression models (via the Service Endpoints created earlier).

Our Certifai Scan needs some mandatory parameters like:

1. Prediction Task Outcomes and Values
2. Model Details (names, endpoints and more)
3. Datasets to evaluate the models on

And optional parameters that depend on the desired evaluation reports. Evaluation types include:

1. Fairness
2. Robustness
3. Explainability

In [12]:
# make sure certifai package was installed correctly
!certifai --version

Certifai version: 1.2.14
Scanner build: 1.2.14-115-g96dc7f43


In [13]:
# necessary imports for creating a scan

from certifai.scanner.builder import (CertifaiScanBuilder, CertifaiModel, CertifaiModelMetric,
                                      CertifaiDataset, CertifaiGroupingFeature, CertifaiDatasetSource,
                                      CertifaiPredictionTask, CertifaiTaskOutcomes, CertifaiOutcomeValue)
from certifai.scanner.report_utils import scores, construct_scores_dataframe


### define cortex certifai task type

- `CertifaiTaskOutcomes` : cortex certifai supports classification as well as regression models. here we have an example of binary-classification (e.g. predict whether loan should be granted or not)
- `CertifaiOutcomeValue` : define the different outcomes possible from the model predictions. here we have a model that predicts either 1(loan granted) or 2(loan denied)

**Note**: Please refer to [Certifai Api Docs](https://cognitivescale.github.io/cortex-certifai/certifai-api-ref/certifai.scanner.builder.html) for more details

In [14]:
# Create the scan object from scratch using the ScanBuilder class with tasks and outcomes

# First define the possible prediction outcomes
task = CertifaiPredictionTask(CertifaiTaskOutcomes.classification(
    [
        CertifaiOutcomeValue(1, name='Loan granted', favorable=True),
        CertifaiOutcomeValue(2, name='Loan denied')
    ]),
    prediction_description='Determine whether a loan should be granted')

#  create a certifai scan object and add the certifai task created above
scan = CertifaiScanBuilder.create('model_auth_demo',
                                  prediction_task=task)


In [15]:
!pip install azure-storage-blob

### Upload dataset to Azure Blob Storage

Get the connection string for the storage account which holds the container called {az_container_name} in the below cell. You can obtain the connection string by following the [Azure guide here](https://docs.microsoft.com/en-us/azure/storage/common/storage-configure-connection-string)

Ensure that the storage account and container used here matches the values you used at the beginning of this notebook to setup your Certifai Pro instance!

In [16]:
# upload our eval dataset to the blob storage container
az_container_name = 'pkandarpa' # this container should already exist. You can create one from the Azure Portal

import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# set credentials for azure storage account
az_credentials = 'REDACTED'


client = BlobServiceClient.from_connection_string(az_credentials)

In [20]:
german_credit_eval_data_file = "data/german_credit_eval.csv"
az_german_credit_blob_name = 'certifai-example/german_credit_eval.csv'

# upload our evaluation dataset to an Azure Blob Storage Account Container.
blob_client = client.get_blob_client(container=az_container_name, blob=az_german_credit_blob_name)
with open(german_credit_eval_data_file, 'rb') as f:
    blob_client.upload_blob(f)


### add logistic and svm models (created above) to scan object

Additional parameters that maybe provided to the `CertifaiModel` class can be gleaned from the [API Reference for CertifaiModel](https://cognitivescale.github.io/cortex-certifai/certifai-api-ref-1.2.14/certifai.scanner.builder.html#certifai.scanner.builder.CertifaiModel)

or `?CertifaiModel`

In [21]:
# Create a Certifai Model Object using the web service (from earlier) by passing the deployed web service url
first_model = CertifaiModel('SVM',
                            predict_endpoint=service_svm_uri)
scan.add_model(first_model)

second_model = CertifaiModel('logistic',
                            predict_endpoint=service_logistic_uri)
scan.add_model(second_model)

# Add corresponding model headers for service authentication and content-type

# add the default headers applicable to all models
scan.add_model_header(header_name='Content-Type',header_value='application/json')

# add defined headers corresponding to auth keys for respective model services
scan.add_model_header(header_name='Authorization', header_value=f'Bearer {service_svm_key}', model_id='SVM')
scan.add_model_header(header_name='Authorization', header_value=f'Bearer {service_logistic_key}', model_id='logistic')



### add the evaluation dataset to scan object

- `evaluation dataset` dataset to be used by cortex certifai to evaluate the model against

In [22]:
# create an evaluation object and pass the evaluation dataset(csv) here 
eval_dataset = CertifaiDataset('evaluation',
                               CertifaiDatasetSource.csv(url=f'abfs://{az_container_name}/{az_german_credit_blob_name}'))
scan.add_dataset(eval_dataset)

### evaluating model fairness 

- add `fairness` as evaluation type to scan object
- create an `evaluation_dataset_id` to refer to added evaluation datset

In [23]:
# Setup an evaluation for fairness on the above dataset using the model
# We'll look at disparity between groups defined by marital status and age
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('age'))
scan.add_fairness_grouping_feature(CertifaiGroupingFeature('status'))
scan.add_evaluation_type('fairness')
scan.evaluation_dataset_id = 'evaluation'

In [24]:
# Because the dataset contains a ground truth outcome column which the model does not
# expect to receive as input we need to state that in the dataset schema (since it cannot
# be inferred from the CSV)
scan.dataset_schema.outcome_feature_name = 'outcome'

### Add Authorization params to our configured models in the scan
We use the following code block to update the scan definition constructed and saved on disk above with authorization headers needed to invoke our Azure Hosted Model Endpoints

In [25]:
local_scan_definition_file = 'data/german_credit_scan_definition.yaml'
model_headers_template = f"""
model_headers:
  default:
  - name: Content-Type
    value: application/json
  - name: accept
    value: application/json
  defined:
  - model_id: SVM
    name: Authorization
    value: Bearer {service_svm_key}
  - model_id: logistic
    name: Authorization
    value: Bearer {service_logistic_key}
"""

with open(local_scan_definition_file, 'w') as f:
    scan.save(f)
# we also need to add the model headers section separately
with open(local_scan_definition_file, 'a') as f:
    f.write(model_headers_template)

## Run a Remote Scan on Certifai Pro

In [26]:
reports_folder = 'abfs://{az_container_name}/certifai-example/reports'
# run a remote scan
!certifai remote scan --alias cpro-az --definition-file data/german_credit_scan_definition.yaml --output {reports_folder}


Created job - certifai-scanner-1948f80c


In [28]:
!certifai remote logs -a cpro-az -n $(certifai remote list -a cpro-az | head -2 | tail -1 | cut -d' ' -f1)


Printing logs for: certifai-scanner-1948f80c-wtb4j

2020-06-03 11:55:53,870 - root - INFO - Validating license...
2020-06-03 11:55:53,871 - root - INFO - License is valid - expires: n/a
2020-06-03 11:55:53,885 - root - INFO - Generated unique scan id: 8565fad160ae
2020-06-03 11:55:53,885 - root - INFO - Validating input data...
2020-06-03 11:55:53,886 - root - INFO - Creating dataset with id: evaluation
2020-06-03 11:55:53,887 - azure.storage.common.storageclient - INFO - Client-Request-ID=2d684c78-a591-11ea-a34a-9203b165f5be Outgoing request: Method=GET, Path=/pkandarpa, Query={'restype': 'container', 'comp': 'list', 'prefix': 'certifai-example', 'delimiter': '/', 'marker': None, 'maxresults': None, 'include': None, 'timeout': None}, Headers={'x-ms-version': '2019-02-02', 'User-Agent': 'Azure-Storage/2.1.0-2.1.0 (Python CPython 3.6.8; Linux 5.3.0-1020-azure)', 'x-ms-client-request-id': '2d684c78-a591-11ea-a34a-9203b165f5be', 'x-ms-date': 'Wed, 03 Jun 2020 11:55:53 GMT', 'Authorizatio

In [29]:
# Check the status of the triggered remote scan job
!certifai remote list -a cpro-az

NAME                        COMPLETIONS   DURATION      AGE           
certifai-scanner-1948f80c   0/1           1m            1m            


## View the reports from this Remote Scan

Once the remote scan's `COMPLETIONS` field says `1/1`, you can configure the Certifai Console to view the reports.

Now, head on over to the URL to the Certifai Console of the Certifai Pro VM instance we created earlier in this tutorial. Use the `User Icon` on the top right and select `Storage Settings` from the dropdown. Update the `Scan Directory` field to the `reports_folder` variable configured in the previous cell. Please omit `abfs://` while pasting this variable's value in the `Scan Directory` field. 

Now, save your settings and wait while the page reloads and loads reports from the remote scan. The scan will be available under the name `model_auth_demo`.


## Resource Cleanup


- Delete the Certifai Pro VM instance created if you are done using it.

ML Pre-requisites Cleanup
 - created and registered `logistic_model_azure` and `svm_model_azure` models to our Azure workspace -> Delete these. The exact names will be available from the output of the `azure_ml_service_keys.py` script. (From when you ran it earlier)
 - created `german-credit-logistic-service` and `german-credit-svm-service` ACI (Azure Container Instance) webservices -> Delete these. The exact names will be available from the output of the `azure_ml_service_keys` script. (From when you ran it earlier)

- Once Cortex Certifai evaluation is complete, make sure to clear all azure resources in order to avoid cost
- Follow the [Azure Ml resource cleanup docs][1] to remove all resources created above

[1]:https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-1st-experiment-sdk-train#clean-up-resources